# Introduction
This notebook explores using the IBM quantum computer library, Qiskit. Qiskit supports running code against one of three Quantum Computers that are available to the  public. It also supports local simulation of quantum circuits.

The main implementation in this notebook is Shor's algorithm. An algorithm for determining the prime factors of an integer.

# Getting started
- In order to get set up with an account, navigate to https://cloud.ibm.com/docs/quantum-computing?topic=quantum-computing-get-started and register a new account.
- Thereafter, create a new quantum instance and generate an API key.

In [1]:
# Import packages

import math
import numpy as np
import random
from qiskit import transpile
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime import QiskitRuntimeService
from math import gcd
import time

Follow the steps in Getting Started to create an IBM account. If you wish to execute locally, and not against an actual Quantum Computer, you can ignore the below cell.

In [35]:
# You can safely ignore this cell if you do not wish to run on a Quantum Computer, as long as you are running locally in the cells that follow

# Try to import the Secrets module which contains sensitive information and use the details to save a Qiskit account locally. This will be used for running on a Quantum Computer.

# To run on a Quantum Computer, ensure that this file has been added and that IBM_QISKIT_API_KEY and IBM_QISKIT_INSTANCE_NAME are variables in it. Alternatively, update the api_key and instance variables below.

try:
    import Secrets
    # Check if the Secrets module contains the required API key and instance name attributes.
    if hasattr(Secrets, "IBM_QISKIT_API_KEY") and hasattr(Secrets, "IBM_QISKIT_INSTANCE_NAME"):
        # Retrieve the API key and service instance name from the Secrets module.
        api_key = Secrets.IBM_QISKIT_API_KEY  # The API key generated from the IBM cloud.
        instance = Secrets.IBM_QISKIT_INSTANCE_NAME  # The name of the IBM Quantum service instance.

        # Save the retrieved account credentials to disk, set it as the default account,
        # and overwrite any previously saved account if it exists.
        QiskitRuntimeService.save_account(
            channel="ibm_cloud",  # Specify the communication channel (IBM Cloud).
            token=api_key,  # Specify the API key for authentication.
            instance=instance,  # Specify the service instance.
            set_as_default=True,  # Set this account as the default account for future use.
            overwrite=True  # Overwrite any existing account credentials with the same name.
        )
    else:
        # If the required attributes (API key or instance name) are missing, display an error message.
        print("Error: The Secrets module is missing required attributes 'IBM_QISKIT_API_KEY' and/or 'IBM_QISKIT_INSTANCE_NAME'.")
except ImportError:
    # If the Secrets module is not found, display an error message.
    print("Error: Secrets file is missing. Please add a Secrets module to the repository.")

Create a sample job that runs an empty circuit and prints a job id. This is used to check whether we are connecting successfully. If you wish to run locally only, ignore this cell.

In [ ]:
# Create a sample job that runs an empty circuit and prints a job id. Used to check whether we are connecting successfully

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# Create empty circuit
example_circuit = QuantumCircuit(2)
example_circuit.measure_all()

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

sampler = Sampler(backend)
job = sampler.run([example_circuit])
print(f"job id: {job.job_id()}")

# Shor's Algorithm
## Introduction
Factorizing N is an important problem in cryptography. Classical public-key cryptography systems,
such as RSA, rely on the fact that factoring large integers (N) into their prime components (p and q
such that $N = p * q$) is computationally hard for classical systems. Shor's algorithm, when implemented
on a quantum computer, provides an efficient solution to this problem, threatening classical cryptographic
security measures.

## Implementation Steps
Shor's algorithm works in the following steps:

Step 1: Choose a suitable value of $a$
- We want to pick a random integer $a$ such that $1 < a < N$ and $gcd(a, N) = 1$ (i.e., $a$ is coprime with $N$).
- This step can be done classically and requires us to use the greatest common divisor (gcd) function to verify co-primality.
- If $\textrm{gcd}(a, N) \neq 1$, then $N$ is not a prime number and we already have a factor of $N$ (this step makes the algorithm probabilistic).

Step 2: Quantum phase estimation to find the period $r$
- Using quantum computation, this step determines the period $r$ of the modular exponentiation function: $a^x mod N$.
- The function $a^x \textrm{mod} N$ is periodic because, for an integer period $r$, we have:
      $a^(x + r) \textrm{mod} N = a^x \textrm{mod} N$
- Quantum phase estimation is central to this step because it leverages the superposition of quantum states to find the period efficiently.
- If this step is run without a quantum computer, it would take exponential time, which is infeasible for large $N$.

Step 3: Use the period $r$ to factorize N
- Once we compute the period $r$, we proceed as follows:
    - If r is odd or if $a^{r/2} \equiv 1 (\textrm{mod} N)$, we cannot reliably factorize $N$, and we may need to try a new value of $a$.
    - Otherwise, compute the factors of N using:
        factors = $\textrm{gcd}(a^{r/2} - 1, N)$ and $\textrm{gcd}(a^{r/2} + 1, N)$
    - One or both of these gcd calculations will yield a factor of N with high probability.
- Mathematically, this step works because the properties of modular arithmetic link the period $r$ to the factors of $N.$

## Considerations
Why is Shor's algorithm probabilistic?
- The success of Shor's algorithm depends on the selection of $a$. If a chosen $a$ does not provide a useful period $r$,
  we must retry with a new value of $a$. This makes the algorithm probabilistic.
- Additionally, quantum computers produce results probabilistically due to the nature of quantum measurements.
  This means that we might have to run the quantum computation multiple times to increase the confidence in the result.

Dependence on quantum computers:
- The critical Step 2 would take exponential time on a classical computer since determining the period $r$ using
  modular exponentiation grows exponentially with the size of N.
- Quantum computers, however, leverage superposition and interference to solve the period-finding problem efficiently,
  making them exponentially faster than classical computers for this specific task.

In [18]:
# Shor's algorithm implementation

def shors_algorithm(N, run_locally):
    """
    Implements Shor's algorithm to find the non-trivial prime factors of a given number.

    Args:
        N (int): The number to factorize.
        run_locally (bool): Whether to run the algorithm locally or on IBM Quantum.

    Returns:
        tuple: A pair of non-trivial factors of 'n'.
    """
    if N % 2 == 0:  # Check if n is even
        return 2  # Return 2 as a factor

    # Pick a random number 'a' in the range [2, n-1]
    a = random.randint(2, N - 1)
    g = gcd(a, N)  # Compute the greatest common divisor (GCD) of a and n

    # If the gcd is not one then return it as a factor
    if g != 1:
        print(f"GCD of {a} and {N} is {g}. Returning {g} as a factor. Did not need to run on quantum computer.")
        return g

    # Perform QPE to find the order 'r' of 'a' modulo 'n'
    if run_locally:
        r = qpe_ibm_local_simulator(a, N)
    else:
        r = qpe_ibm_quantum_backend(a, N)

    if r % 2 != 0 or pow(a, r // 2, N) == N - 1:  # Check if r is odd or invalid for factorization
        return shors_algorithm(N, run_locally)  # Retry with a different random 'a'

    # Compute the factors using the order 'r'
    factor1 = gcd(pow(a, r // 2) - 1, N)
    factor2 = gcd(pow(a, r // 2) + 1, N)

    if factor1 == 1 or factor2 == 1:  # If factors are trivial, retry
        return shors_algorithm(N, run_locally)

    return factor1, factor2  # Return the non-trivial factors of n


def modular_exponentiation(a, x, N, n):
    """
    Creates a quantum circuit instruction to perform modular exponentiation.

    Args:
        a (int): The base of the exponentiation.
        x (int): The power to which the base is raised.
        N (int): The modulus.
        n (int): Number of qubits required to represent the modulus.

    Returns:
        Instruction: A quantum circuit instruction for modular exponentiation.
    """
    qc = QuantumCircuit(n + 1)
    for i in range(n):
        qc.p(2 * np.pi * (a**(x % N)) / N, i)  # Phase rotation with modular exponentiation
    return qc.to_instruction()  # Return the quantum instruction to be reused


def inverse_qft(n):
    """
    Constructs an inverse Quantum Fourier Transform (QFT) circuit.

    Args:
        n (int): The number of qubits in the circuit.

    Returns:
        Instruction: A quantum circuit instruction for the inverse QFT.
    """
    qc = QuantumCircuit(n)
    for i in range(n // 2):
        qc.swap(i, n - i - 1)  # Swap qubits to reverse their order
    for i in range(n):
        for j in range(i):
            qc.cp(-np.pi / 2**(i - j), j, i)  # Controlled phase gate
        qc.h(i)  # Apply Hadamard gate
    return qc.to_instruction()  # Return the quantum instruction to be reused


def find_order_from_phase(phase_decimal):
    """
    Determines the order from the phase result of Quantum Phase Estimation (QPE).

    Args:
        phase_decimal (float): The phase result as a decimal value.

    Returns:
        int: The denominator of the phase fraction, which is the order.
    """
    fraction = phase_decimal.as_integer_ratio()  # Convert the phase into a fraction
    return fraction[1]  # The denominator gives the order


def qpe_setup(a, N, max_available_qubits=math.inf):
    """
    Generates a quantum circuit for the generalized phase estimation setup.

    Args:
        a (int): The base for modular exponentiation.
        N (int): The modulus.

    Returns:
        tuple: A tuple consisting of the quantum circuit (QuantumCircuit) and the
               number of phase estimation qubits (int).
               :param max_available_qubits:
    """
    n = math.ceil(math.log2(N))  # Number of qubits needed to represent N
    m = 2 * n  # Extra qubits for accuracy in the QPE

    #if m > max_available_qubits:
    #    raise ValueError(f"Need {max_available_qubits} qubits for the quantum circuit but only have {m} available.")

    qc = QuantumCircuit(m + n, m)  # Create a circuit with m phase qubits and n target qubits

    qc.x(m)  # Set the first qubit of the target register to |1>

    # Initialize the phase qubits in superposition
    qc.h(range(m))  # Apply Hadamard gates to phase qubits

    # Apply controlled unitary operations
    for qubit in range(m):
        qc.append(modular_exponentiation(a, 2**qubit, N, n),
                  [qubit] + list(range(m, m + n)))  # Controlled evolution

    # Apply the inverse QFT to the phase register
    qc.append(inverse_qft(m), range(m))

    # Measure the phase qubits
    qc.measure(range(m), range(m))

    return qc, m


def qpe_ibm_quantum_backend(a, N, time_out=15):
    """
    Implements Quantum Phase Estimation (QPE) to estimate the phase and determine the order using IBM's quantum computers.

    Args:
        a (int): The base for modular exponentiation.
        N (int): The modulus.
        time_out: The timeout in seconds for the job on the IBM Quantum computer. Default is 15 seconds.

    Returns:
        int: The order 'r' of the modular exponentiation.
    """

    print("Performing quantum phase estimation on IBM Quantum Computer...")

    # Connect to IBM Quantum using Qiskit Runtime
    print("Connecting to IBM Quantum Computer...")
    start_time = time.time()
    service = QiskitRuntimeService()
    backend = service.least_busy(operational=True, simulator=False)
    execution_time = time.time() - start_time
    print(f"Connected to {backend}. Took {execution_time} seconds")

    # Access the number of qubits from the backend
    num_qubits = backend.num_qubits
    print(f"Number of qubits on quantum computer: {num_qubits}")

    # Create the quantum circuit
    qc, m = qpe_setup(a, N, num_qubits)

    # Run using the Sampler primitive
    transpiled_qc = transpile(qc, backend)
    sampler = SamplerV2(backend)
    start_time = time.time()
    print("Starting job on Quantum Computer...")
    job = sampler.run([transpiled_qc])
    result = job.result(timeout=time_out)
    execution_time = time.time() - start_time
    print(f"Execution time of job on Quantum Computer: {execution_time} seconds")

    # Extract phase estimation result using join_data and get_counts
    pub_result = result[0]
    counts = pub_result.join_data().get_counts()
    measured_value = max(counts, key=counts.get)
    phase_estimate = int(measured_value, 2) / (2**m)
    r = find_order_from_phase(phase_estimate)

    return r


def qpe_ibm_local_simulator(a, N):
    """
    Implements Quantum Phase Estimation (QPE) to estimate the phase and determine the order using a local simulator.

    Args:
        a (int): The base for modular exponentiation.
        N (int): The modulus.

    Returns:
        int: The order 'r' of the modular exponentiation.
    """

    # Create the quantum circuit
    qc, m = qpe_setup(a, N)

    simulator = AerSimulator()  # Use Qiskit's AerSimulator
    transpiled_qc = transpile(qc, simulator)  # Transpile the circuit for the simulator
    job = simulator.run(transpiled_qc)  # Run the simulation
    result = job.result()  # Get the result object

    # Extract phase estimation result
    counts = result.get_counts()
    measured_value = max(counts, key=counts.get)
    phase_estimate = int(measured_value, 2) / (2**m)
    r = find_order_from_phase(phase_estimate)

    return r


# Running Shor's Algorithm
In the below cell, run Shor's algorithm using 'run_locally' (to specify whether the simulation should be performed locally, and not on the quantum computer) and 'N', the integer to factorize

In [32]:
run_locally = True
N = 15

if run_locally:
    print(f"Running Shor's algorithm locally to find the non-trivial prime factors of {N}.")
    result = shors_algorithm(15, True)
    print(f"The non-trivial prime factors of {N} are {result}.")
else:
    print(f"Running Shor's algorithm on quantum computer to find the non-trivial prime factors of {N}.")

    result = shors_algorithm(N, False)
    print(f"The non-trivial prime factors of {N} are {result}.")


Running Shor's algorithm locally to find the non-trivial prime factors of 15.
The non-trivial prime factors of 15 are 5.


Further testing

In [34]:
# Run Shor's algo locally on a range of numbers and test against expected factors to see whether valid factors were found and/or whether any cases of factors were found that were in fact not

from itertools import product
import sympy

def get_factors_from_prime_factors(prime_factors):
    """
    Get all unique factors of a number using its prime factorization.

    Args:
        prime_factors (dict): A dictionary where each key is a prime number and its value is the corresponding exponent.
                              Example: For 16 (2^4), the input should be {2: 4}.

    Returns:
        List[int]: A sorted list of all possible factors of the number created from the prime factors.
    """
    # Generate a range of powers for each prime factor from 0 to its exponent
    factor_ranges = [[p ** e for e in range(exp + 1)] for p, exp in prime_factors.items()]

    # Generate all possible combinations of the factors using Cartesian product
    all_factors = set(map(lambda x: eval('*'.join(map(str, x))), product(*factor_ranges)))

    # Sort and return the list of factors
    return sorted(all_factors)

def get_all_factors(n):
    """
    Compute all factors of a number using its prime factorization.

    Args:
        n (int): The number to compute the factors for.

    Returns:
        List[int]: A list of all unique factors of the input number, sorted in ascending order.
    """
    # Factorize the number into its prime factors using sympy
    sympy_factors = sympy.factorint(n)

    # Use the prime factorization to compute all factors
    return get_factors_from_prime_factors(sympy_factors)

# Flags to track issues in the algorithm
any_non_actual_factors_found = False  # Tracks if any found factors are incorrect
any_case_of_no_factors_found = False  # Tracks if factors could not be found for any number

# Loop through numbers from 2 to 99 and verify Shor's algorithm results
start = 2
end = 100
for i in range(start, end+1):
    actual_factors = get_all_factors(i)  # Get all actual factors for the number 'i'

    # Skip numbers that are prime (since they have exactly two factors: 1 and themselves)
    if len(actual_factors) == 2:
        continue

    # Run Shor's algorithm to compute the factors of 'i' locally
    shor_factors = shors_algorithm(i, True)

    if shor_factors is None:
        # If Shor's algorithm fails to find the factors, log the issue
        any_case_of_no_factors_found = True
        print("No factors found for", i)
    elif isinstance(shor_factors, tuple):
        # If Shor's algorithm returns a tuple of factors, check each factor
        print(f"Tuple found for {i}: {shor_factors}")
        for f in shor_factors:
            if f not in actual_factors:
                # Log any incorrect factors
                print(f"Factor {f} not found in actual factors {actual_factors} for {i}")
    else:
        # If Shor's algorithm returned a single factor, verify its correctness
        if shor_factors not in actual_factors:
            any_non_actual_factors_found = True
            print(f"Factor {shor_factors} not found in actual factors {actual_factors} for {i}")

# Print a summary of the findings
if any_non_actual_factors_found:
    print("Problem detected. Non-actual factors found using Shor's algorithm.")

if any_case_of_no_factors_found:
    print("Problem detected. No factors found using Shor's algorithm.")

if not any_case_of_no_factors_found and not any_non_actual_factors_found:
    print(f"No issues found when executing Shor's algorithm on all numbers in range {start} to {end}.")